In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Inference with a panoptic segmentation model
cfg = get_cfg()

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join('/media/prakash/LaCie1/Backup/opendet2_aimlteam/output/Output_files_k9s_4gpus/faster_rcnn_Swin_T_FPN_3x_opendet_specnorm_4gpu/model_0059999.pth')  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.structures import BoxMode

def get_balloon_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "val"]:
    DatasetCatalog.register("balloon_" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
    MetadataCatalog.get("balloon_" + d).set(thing_classes=["balloon"])
balloon_metadata = MetadataCatalog.get("balloon_train")

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_balloon_dicts("balloon/val")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    image = out.get_image()[:, :, ::-1]  # Convert to BGR format if needed
    # Specify the file path where you want to save the image
    file_path = "output_image.jpg"  # Change the file extension as needed (e.g., .png, .jpeg)

    # Save the image to the specified file
    cv2.imwrite(file_path, image)
    #cv2.imshow("Image Window", image)

In [93]:
import cv2
import numpy as np

# Function to parse the prediction file
def parse_prediction_file(file_path):
    predictions = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line:
                parts = line.split()
                image_id = parts[0]
                confidence = float(parts[1])
                bbox = [float(parts[i]) for i in range(2, 6)]  # Use float instead of int for bounding box coordinates
                predictions.append((image_id, confidence, bbox))
    return predictions

# Function to save the inference results of a specific image ID with bounding boxes
def save_results(image_path, predictions, target_image_id, output_image_path):
    image = cv2.imread(image_path)
    
    for image_id, confidence, bbox in predictions:

        if int(image_id) == target_image_id:
            # Draw bounding box on the image
            x, y, w, h = map(int, bbox)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f"Confidence: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imwrite(output_image_path, image)


if __name__ == "__main__":
    
    prediction_file = '/media/prakash/LaCie1/Backup/opendet2/output/faster_rcnn_Swin_T_FPN_3x_opendet_with_specnorm/pascal_voc_eval/unknown.txt'  # Replace with the path to your prediction file
    #prediction_file = '/media/prakash/LaCie1/Backup/opendet2/output/faster_rcnn_R_50_FPN_3x_opendet_original/pascal_voc_eval/unknown.txt'  # Replace with the path to your prediction file
    image_id_to_visualize = 9210 # Replace with the specific image ID you want to visualize
    image_path = format('/media/prakash/LaCie1/Backup/opendet2/datasets/voc_coco/JPEGImages/00%s.jpg' %(image_id_to_visualize) )  # Replace with the path to the corresponding image
    
    output_image_path = 'output_image.jpg'  # Replace with the desired output image file path
    
    predictions = parse_prediction_file(prediction_file)
    save_results(image_path, predictions, image_id_to_visualize, output_image_path)


